# Mining Processes From Smart Contracts

In [ ]:
from web3 import Web3
w3 = Web3(Web3.WebsocketProvider("ws://127.0.0.1:8545", websocket_kwargs={'timeout': 60}))
w3.isConnected()

In [ ]:
CONTRACT_ADDRESS_A = "0x54497Da55F80B125a2050373fb7230e5A4eFA077"
CONTRACT_ADDRESS_B = "0x7F79072c37A02e1F9474aa18Be0B4FeAA68cE624"
PARTICIPANTS_CONTAINER = "0xcd23eCBBdBcC03aE1Bcf658Fd3893F2c6f87DeEf"


block = w3.eth.getCode(CONTRACT_ADDRESS_A)
contract = w3.eth.contract(CONTRACT_ADDRESS_A)

#print(block)
#print(contract)

contract_a_filter = w3.eth.filter({
    "fromBlock": "0x0",
    "toBlock": "latest",
    "address": CONTRACT_ADDRESS_A
})

contract_a_eventlist = contract_a_filter.get_all_entries()
print(contract_a_eventlist)
print("###########\n\n")


contract_b_filter = w3.eth.filter({
    "fromBlock": "0x0",
    "toBlock": "latest",
    "address": CONTRACT_ADDRESS_B
})

contract_b_eventlist = contract_b_filter.get_all_entries()
print(contract_b_eventlist)
print("###########\n\n")


participants_container_filter = w3.eth.filter({
    "fromBlock": "0x0",
    "toBlock": "latest",
    "address": PARTICIPANTS_CONTAINER
})

participants_container_eventlist = participants_container_filter.get_all_entries()
print(participants_container_eventlist)